# Corrigé : Premières manipulations

Quelques élément de corrigé

In [ ]:
import pandas

In [ ]:
prénom = pandas.read_csv("nat2022_csv.zip",
                         compression='zip',
                         sep=";")

In [ ]:
# prénom = pandas.read_csv("https://www.insee.fr/fr/statistiques/fichier/7633685/nat2022_csv.zip",
#                          compression='zip',
#                          sep=";")

## Votre prénom

### étude d'un prénom

On utilise le cours 1.1

### graphique

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

sns.reset_orig() # reset au cas où on a déjà exécuté des cellules en-dessous
sns.set()
color_palette = sns.color_palette()


#### François

Les François homme par an :

In [ ]:
data = (prénom[(prénom["preusuel"] == 'FRANÇOIS') & (prénom["sexe"] == 1)]
            .sort_values(by=['annais']))

data

In [ ]:
# 1. créer le dessin (ici ax)
fig, ax = plt.subplots(figsize=(20, 5)) 

#  2. ajouter des choses au dessin
chart = sns.lineplot(x=data.annais, 
                     y=data.nombre,
                     color=color_palette[0],
                     ax=ax)

ax.axhline(data.nombre.mean(), ls='--', color=color_palette[1])
ax.axvline(x='1981', ls='--', color=color_palette[2])
ax.axvline(x='2012', ls='--', color=color_palette[2])


plt.xticks(rotation=90)
plt.title("nombre de François ♂ par année")
plt.xlabel('année')

# 3. représenter le graphique
plt.show()

#### Les Adolphe

In [ ]:
data = (prénom[(prénom["preusuel"] == 'ADOLPHE') & (prénom["sexe"] == 1)]
            .sort_values(by=['annais']))


Attention aux années sans Adolphe :

In [ ]:
data.tail()

On ne peut donc pas faire comme avant. Il faut ajouter des lignes pour les années manquantes.

On procéde comme dans le cours avec les françois filles :

In [ ]:
data = (prénom[(prénom["preusuel"] == 'ADOLPHE') & (prénom["sexe"] == 1) & (prénom['annais'] != "XXXX")]
            .assign(annais=lambda df: pandas.to_numeric(df.annais))
            .drop(["sexe", "preusuel"], axis=1)
            .set_index("annais")
            .reindex(pandas.RangeIndex(1900,2021 + 1,1))
            .fillna(0)
            .sort_index()
)
data

In [ ]:
# 1. créer le dessin (ici ax)
fig, ax = plt.subplots(figsize=(20, 5)) 

#  2. ajouter des choses au dessin
chart = sns.lineplot(x=data.index, 
                     y=data.nombre,
                     color=color_palette[0],
                     ax=ax)

ax.axhline(data.nombre.mean(), ls='--', color=color_palette[1])

plt.xticks(rotation=90)
plt.title("nombre de Adolphe ♂ par année")
plt.xlabel('année')

# 3. représenter le graphique
plt.show()

### de l'importance des séries dans la vie

TBD

## Répartition des prénoms

In [ ]:
dominique_1 = (prénom[(prénom["preusuel"] == "DOMINIQUE") & (prénom["sexe"] == 1)]
                   .drop(["preusuel", "sexe"], axis=1)
                   .sort_values(by=['annais']))

dominique_2 = (prénom[(prénom["preusuel"] == "DOMINIQUE") & (prénom["sexe"] == 2)]
                   .drop(["preusuel", "sexe"], axis=1)
                   .sort_values(by=['annais']))

In [ ]:
dominiques = (pandas
                 .merge(dominique_1, dominique_2, on=['annais'], how='outer')
                 .fillna(0)
                 .rename(columns={"nombre_x": "♂", "nombre_y": "♀"})
            )

In [ ]:
dominiques

### graphiques

On utilise le 1.2, puis on ajoute une colonne de pourcentage. Ici le pourcentage d'hommes

In [ ]:
dominiques = (dominiques
                  .assign(pourcent=dominiques["♂"] / (dominiques["♂"] + dominiques["♀"]))
                  .drop(120))

In [ ]:
dominiques

In [ ]:
data = dominiques

fig, ax = plt.subplots(figsize=(20, 5)) 

ax.set_ylim(-0.1, 1.1)

chart = sns.lineplot(x=data.annais, 
                     y=data['pourcent'],
                     color=color_palette[0],
                     ax=ax)

ax.axhline(0.5, ls='--', color=color_palette[1])

plt.xticks(rotation=90)
plt.title("pourçentage de Dominique ♂ par année")
plt.xlabel('année')

plt.show()

## autres études

### Prénoms mixtes

In [ ]:
# les prénoms portés par des garçons :

prénom[prénom.sexe == 1].preusuel

In [ ]:
# liste unique

prénom[prénom.sexe == 1].preusuel.unique()

On fait l'instersection des deux avec des [ensembles](https://docs.python.org/3/tutorial/datastructures.html#sets) :

In [ ]:
mixte = set(prénom[prénom.sexe == 1].preusuel.unique()) & set(prénom[prénom.sexe == 2].preusuel.unique())

In [ ]:
mixte

In [ ]:
mixte.remove("_PRENOMS_RARES")

In [ ]:
len(mixte)

In [ ]:
"FRANÇOIS" in mixte

### Mode 

In [ ]:
(prénom[(prénom.annais == '1900') & (prénom.sexe == 1)]
    ['nombre']
     .sum()
)

In [ ]:
(prénom[(prénom.annais == '1900') & (prénom.sexe == 1)]
     ['nombre']
     .idxmax()
)

In [ ]:
prénom.iloc[134841]

In [ ]:
max_homme = pandas.DataFrame(index=pandas.RangeIndex(1900,2021 + 1,1))

max_homme = (max_homme
    .assign(total=[prénom[(prénom.annais == str(idx)) & (prénom.sexe == 1)]['nombre'].sum() for idx in max_homme.index])
    .assign(prénom=[prénom.iloc[prénom[(prénom.annais == str(idx)) & (prénom.sexe == 1)]['nombre'].idxmax()]['preusuel'] for idx in max_homme.index])
)

In [ ]:
max_homme = (max_homme
     .assign(nombre=lambda df:[prénom[(prénom['annais'] == str(idx)) & (prénom['preusuel']== df.loc[idx]['prénom'])]['nombre'].sum() for idx in df.index])
     .assign(pourcent=lambda df: df['nombre'] / df['total'])
)

In [ ]:
max_homme['prénom'].unique()

Sans les prénoms rares :

In [ ]:
max_homme2 = pandas.DataFrame(index=pandas.RangeIndex(1900,2021 + 1,1))
            
max_homme2 = (max_homme2
    .assign(total=[prénom[(prénom.annais == str(idx)) & (prénom.sexe == 1)]['nombre'].sum() for idx in max_homme.index])
    .assign(prénom=[prénom.iloc[prénom[(prénom.annais == str(idx)) & (prénom.sexe == 1) & (prénom.preusuel != '_PRENOMS_RARES')]['nombre'].idxmax()]['preusuel'] for idx in max_homme.index])
    .assign(nombre=lambda df:[prénom[(prénom['annais'] == str(idx)) & (prénom.sexe == 1) & (prénom['preusuel'] == df.loc[idx]['prénom'])]['nombre'].sum() for idx in df.index])
    .assign(pourcent=lambda df: df['nombre'] / df['total'])
)

max_homme2

In [ ]:
max_homme2['prénom'].unique()

In [ ]:
max_fille2 = pandas.DataFrame(index=pandas.RangeIndex(1900,2021 + 1,1))

max_fille2 = (max_fille2              
    .assign(total=[prénom[(prénom.annais == str(idx)) & (prénom.sexe == 2)]['nombre'].sum() for idx in max_homme.index])
    .assign(prénom=[prénom.iloc[prénom[(prénom.annais == str(idx)) & (prénom.sexe == 2) & (prénom.preusuel != '_PRENOMS_RARES')]['nombre'].idxmax()]['preusuel'] for idx in max_homme.index])
    .assign(nombre=lambda df:[prénom[(prénom['annais'] == str(idx)) & (prénom.sexe == 2) & (prénom['preusuel'] == df.loc[idx]['prénom'])]['nombre'].sum() for idx in df.index])
    .assign(pourcent=lambda df: df['nombre'] / df['total'])
)

max_fille2

In [ ]:
max_fille2['prénom'].unique()

### sexe par année

In [ ]:
années = [str(x) for x in range(1900, 2021 + 1)]
pourcent = [0] * len(années)
homme = [0] * len(années)
femme = [0] * len(années)

In [ ]:
repartition = pandas.DataFrame({"pourcent":pourcent, "homme":homme, "femme": femme}, index=années)

Attention au type : https://www.geeksforgeeks.org/change-data-type-for-one-or-more-columns-in-pandas-dataframe/

Si vous ne faites pas attention vous aurez des entiers et pas des réels

In [ ]:
repartition.dtypes

In [ ]:
repartition.loc["1900", "pourcent"] = 0.5

In [ ]:
repartition.loc[["1900"]]

In [ ]:
repartition = repartition.astype({"pourcent":float})

In [ ]:
repartition.dtypes

Le calcul suivant est très long. On doit pouvoir faire (bien) mieux. Je laisse cependant cette approche pour vous montrer comment on peut tout de même *bourriner* les calcul si on ne connais pas toutes les méthodes. C'est juste bien plus long.

In [ ]:
for annee in repartition.index:
    
    homme = sum(prénom[(prénom.sexe == 1) & (prénom.annais == annee)]['nombre'])
    femme = sum(prénom[(prénom.sexe == 2) & (prénom.annais == annee)]['nombre'])

    repartition.loc[annee, "pourcent"] = homme / (homme + femme)
    repartition.loc[annee, "homme"] = homme
    repartition.loc[annee, "femme"] = femme

In [ ]:
repartition

In [ ]:
data = repartition

fig, ax = plt.subplots(figsize=(20, 5)) 

chart = sns.lineplot(x=data.index, 
                     y=data['pourcent'],
                     ax=ax)

plt.xticks(rotation=90)
plt.title("pourçentage d'homme par année'")
plt.xlabel('année')


plt.show()

In [ ]:
années = [str(x) for x in range(1900, 2021 + 1)]
pourcent = [0.0] * len(années)
homme = [0] * len(années)
femme = [0] * len(années)

repartition2 = pandas.DataFrame({"pourcent":pourcent, "homme":homme, "femme": femme}, index=années)

for année in repartition2.index:
    
    homme = sum((prénom.sexe == 1) & (prénom.annais == année))
    femme = sum((prénom.sexe == 2) & (prénom.annais == année))

    repartition2.loc[année, "pourcent"] = homme / (homme + femme)
    repartition2.loc[année, "homme"] = homme
    repartition2.loc[année, "femme"] = femme

In [ ]:
repartition2

In [ ]:
data = repartition2

fig, ax = plt.subplots(figsize=(20, 5)) 

chart = sns.lineplot(x=data.index, 
                     y=data['pourcent'],
                     ax=ax)

plt.xticks(rotation=90)
plt.title("pourçentage de nombre différent de prénoms homme par année'")
plt.xlabel('année')


plt.show()

### coefficient de Gini

**TBD**